In [1]:
!pip install python-docx transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.5 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()


Saving final_user_stories.docx to final_user_stories.docx


In [3]:
import docx
import json
import re

# Load the Word file
doc = docx.Document("final_user_stories.docx")

# Combine all text from all paragraphs
raw_text = "\n".join([para.text.strip() for para in doc.paragraphs if para.text.strip()])

# Split into entries (user story + JSON block)
entries = re.split(r'(?<=\})\s*(?=As a )', raw_text)

valid_data = []
invalid_entries = []

for entry in entries:
    try:
        user_story_part = entry[:entry.index('{')].strip()
        json_part = entry[entry.index('{'):].strip()
        parsed_json = json.loads(json_part)

        valid_data.append({
            "user_story": user_story_part,
            "uml_json": parsed_json
        })
    except Exception as e:
        invalid_entries.append({
            "entry": entry,
            "error": str(e)
        })

# Save cleaned valid entries
with open("cleaned_dataset.jsonl", "w") as f:
    for row in valid_data:
        f.write(json.dumps(row) + "\n")

# Save bad ones to review manually
with open("invalid_entries.txt", "w") as f:
    for row in invalid_entries:
        f.write("Error: " + row["error"] + "\n")
        f.write(row["entry"] + "\n\n")

print(f"✅ Valid Entries: {len(valid_data)}")
print(f"❌ Invalid Entries: {len(invalid_entries)}")


✅ Valid Entries: 245
❌ Invalid Entries: 23


In [4]:
import pandas as pd

# Load JSONL
data = []
with open("cleaned_dataset.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

# Define keyword-label map
keyword_map = {
    "exercise": "Fitness", "steps": "Fitness", "calories": "Fitness",
    "task": "Task Management", "deadline": "Task Management", "freelancer": "Task Management",
    "invoice": "Billing", "payment": "Payments", "order": "Ordering",
    "menu": "Food", "nutrition": "Food",
    "review": "Reviews", "rating": "Reviews",
    "product": "E-commerce", "cart": "E-commerce",
    "track": "Tracking", "goal": "Fitness",
    "course": "Education", "student": "Education",
    "event": "Events", "profile": "Account", "account": "Account",
    "login": "Authentication", "auth": "Authentication",
    "bill": "Payments", "discount": "Promotions", "promotion": "Promotions"
}

def assign_label(text):
    t = text.lower()
    for k, v in keyword_map.items():
        if k in t:
            return v
    return "Other"

df["label"] = df["user_story"].apply(assign_label)
df.to_csv("labeled_dataset.csv", index=False)
print("✅ Saved as labeled_dataset.csv")


✅ Saved as labeled_dataset.csv


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the labeled dataset
df = pd.read_csv("labeled_dataset.csv")

# ✅ Filter out labels with less than 2 examples
label_counts = df["label"].value_counts()
valid_labels = label_counts[label_counts >= 2].index
df = df[df["label"].isin(valid_labels)]

# ✅ Split into train/val
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["user_story"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# ✅ Encode the labels to numbers
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)

# ✅ See what's left
print("🎯 Classes after filtering:\n", df["label"].value_counts())
print("✅ Split sizes: ", len(train_texts), "train /", len(val_texts), "val")


🎯 Classes after filtering:
 label
Other              124
Account             23
Tracking            21
Reviews             10
E-commerce          10
Ordering             9
Task Management      9
Events               8
Payments             8
Food                 6
Billing              6
Fitness              5
Education            5
Name: count, dtype: int64
✅ Split sizes:  195 train / 49 val


In [6]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
import torch

class UserStoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = UserStoryDataset(train_encodings, train_labels)
val_dataset = UserStoryDataset(val_encodings, val_labels)


In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_encoder.classes_),
    id2label={i: label for i, label in enumerate(label_encoder.classes_)},
    label2id={label: i for i, label in enumerate(label_encoder.classes_)}
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # You can use "eval_strategy" if "evaluation_strategy" breaks
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/tmp/ipython-input-21-3027770107.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.899022
2,No log,1.798135
3,No log,1.766410


TrainOutput(global_step=75, training_loss=1.957108154296875, metrics={'train_runtime': 9.0609, 'train_samples_per_second': 64.563, 'train_steps_per_second': 8.277, 'total_flos': 6963665981220.0, 'train_loss': 1.957108154296875, 'epoch': 3.0})

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [22]:
results = trainer.evaluate()
print("📊 Final Evaluation:", results)


📊 Final Evaluation: {'eval_loss': 1.76641047000885, 'eval_runtime': 0.0991, 'eval_samples_per_second': 494.359, 'eval_steps_per_second': 70.623, 'epoch': 3.0}


In [25]:
df = pd.read_csv("labeled_dataset.csv")

# Drop some "Other" entries randomly to balance things a bit
df_balanced = pd.concat([
    df[df["label"] != "Other"],
    df[df["label"] == "Other"].sample(n=30, random_state=42)  # keep only 30 "Other"
])

df_balanced["label"].value_counts()


,count
label,
Other,30
Account,23
Tracking,21
Reviews,10
E-commerce,10
Ordering,9
Task Management,9
Payments,8
Events,8


In [26]:
import pandas as pd

# Load your newly balanced dataset
df = pd.read_csv("labeled_dataset.csv")

# Reduce "Other" to 30 only
df_other = df[df["label"] == "Other"].sample(n=30, random_state=42)
df_rest = df[df["label"] != "Other"]
df = pd.concat([df_other, df_rest])

# Drop classes with fewer than 2 samples
label_counts = df["label"].value_counts()
valid_labels = label_counts[label_counts >= 2].index
df = df[df["label"].isin(valid_labels)]

print("✅ Final label distribution:")
print(df["label"].value_counts())

# Save it
df.to_csv("cleaned_balanced_dataset.csv", index=False)


✅ Final label distribution:
label
Other              30
Account            23
Tracking           21
Reviews            10
E-commerce         10
Ordering            9
Task Management     9
Events              8
Payments            8
Food                6
Billing             6
Fitness             5
Education           5
Name: count, dtype: int64


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load cleaned & balanced data
df = pd.read_csv("cleaned_balanced_dataset.csv")

# Split dataset (stratified)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["user_story"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# Encode labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)

# Save for inference mapping later
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {i: label for label, i in label2id.items()}


In [28]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenize
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Dataset class
class UserStoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = UserStoryDataset(train_encodings, train_labels)
val_dataset = UserStoryDataset(val_encodings, val_labels)


In [29]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-29-1007094520.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.544100,2.365676
2,2.290200,2.275094
3,2.198700,2.120637
4,1.966900,2.015041
5,1.834000,1.968071


TrainOutput(global_step=75, training_loss=2.180298105875651, metrics={'train_runtime': 27.8494, 'train_samples_per_second': 21.544, 'train_steps_per_second': 2.693, 'total_flos': 6521158778400.0, 'train_loss': 2.180298105875651, 'epoch': 5.0})

In [47]:
# 🧱 Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from google.colab import files

# 📤 Step 1: Upload the new_data.csv file manually
uploaded = files.upload()  # 📎 Upload box will appear

# 🧹 Step 2: Load existing and new data
df1 = pd.read_csv("cleaned_balanced_dataset.csv")
df2 = pd.read_csv("new_data.csv")  # make sure you upload it with exact filename

# 🌀 Step 3: Merge and shuffle
df = pd.concat([df1, df2], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

# 🧠 Step 4: Split data (stratified)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["user_story"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# 🏷️ Step 5: Encode labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {i: label for label, i in label2id.items()}

# 🧪 Step 6: Tokenizer
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# 📦 Step 7: Dataset class
class UserStoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self,):
        return len(self.labels)

train_dataset = UserStoryDataset(train_encodings, train_labels)
val_dataset = UserStoryDataset(val_encodings, val_labels)

# 🧠 Step 8: Model
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# ⚙️ Step 9: Training args
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# 🏋️ Step 10: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# 🚀 Step 11: Train
trainer.train()

Saving manual_label_dataset.csv to manual_label_dataset.csv


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-47-3936834584.py:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.527900,2.345685
2,2.195100,2.097185
3,1.835900,1.867205
4,1.659700,1.728890
5,1.485900,1.671738


TrainOutput(global_step=85, training_loss=1.9318499509026021, metrics={'train_runtime': 93.3302, 'train_samples_per_second': 7.286, 'train_steps_per_second': 0.911, 'total_flos': 8094517721760.0, 'train_loss': 1.9318499509026021, 'epoch': 5.0})

In [48]:
results = trainer.evaluate()
print("📊 Final Evaluation Metrics:", results)


📊 Final Evaluation Metrics: {'eval_loss': 1.6717381477355957, 'eval_runtime': 0.0717, 'eval_samples_per_second': 474.45, 'eval_steps_per_second': 69.772, 'epoch': 5.0}


In [49]:
model_path = "./fine_tuned_user_story_classifier"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print("✅ Model saved to", model_path)


✅ Model saved to ./fine_tuned_user_story_classifier


In [51]:
import torch

# 🔥 Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 🧪 Test inference text
test_text = "As a user, I want to invest in stocks and track my returns."
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True)

# ⚙️ Move inputs to same device as model
inputs = {key: val.to(device) for key, val in inputs.items()}

# 🔮 Get model prediction
with torch.no_grad():
    outputs = model(**inputs)

# 🧠 Decode prediction
predicted_class = torch.argmax(outputs.logits, dim=1).item()
predicted_label = label_encoder.inverse_transform([predicted_class])[0]

print("📌 Predicted Label:", predicted_label)


📌 Predicted Label: Tracking


In [52]:
model.save_pretrained("my_distilbert_model")
tokenizer.save_pretrained("my_distilbert_model")


('my_distilbert_model/tokenizer_config.json',
 'my_distilbert_model/special_tokens_map.json',
 'my_distilbert_model/vocab.txt',
 'my_distilbert_model/added_tokens.json',
 'my_distilbert_model/tokenizer.json')

In [53]:
def predict_user_story(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder.inverse_transform([predicted_class])[0]


In [57]:
from google.colab import files
uploaded = files.upload()



Saving manual_label_dataset.csv to manual_label_dataset (1).csv


In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# 👓 Load your manually labeled dataset
df = pd.read_csv("manual_label_dataset.csv")  # <- rename this if the name is different

# 🔍 Basic check
print("✅ Loaded rows:", len(df))
print("📊 Label distribution:\n", df['label'].value_counts())

# 🎯 Prepare text and labels
texts = df["user_story"].tolist()
labels = df["label"].tolist()

# 🎨 Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Save mappings for inference
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {i: label for label, i in label2id.items()}

# 🧪 Split into train/val
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, encoded_labels, test_size=0.2, stratify=encoded_labels, random_state=42
)

# 🧼 Tokenize
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# 🧾 Dataset class
class UserStoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = UserStoryDataset(train_encodings, train_labels)
val_dataset = UserStoryDataset(val_encodings, val_labels)

# 🧠 Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# 🛠️ Training args
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none"  # disables wandb issue
)

# 🧪 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# 🚀 Train
trainer.train()


✅ Loaded rows: 245
📊 Label distribution:
 label
Task Management      25
Banking              24
E-commerce           23
Fitness              15
Other                12
Parking              12
News                 11
Social Media         10
Books                10
Weather              10
\tJobs                9
RentalCars            9
Travel                8
Education             7
Jobs                  7
\tTask Management     6
Events                6
Music                 5
Billing               4
Reviews               4
 Other                4
Accounts              4
Ordering              3
\tE-commerce          3
Promotions            3
Account               2
Other                 2
Task Management       2
Tracking              1
E-commerce            1
Payments              1
Food                  1
Accountt              1
Name: count, dtype: int64


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [59]:
# Count labels
label_counts = df['label'].value_counts()

# Keep only those with at least 2 samples
valid_labels = label_counts[label_counts >= 2].index
df = df[df['label'].isin(valid_labels)]


In [60]:
df = pd.read_csv("manual_label_dataset.csv")

# 💥 Filter labels with <2 samples (stratify-safe)
label_counts = df['label'].value_counts()
valid_labels = label_counts[label_counts >= 2].index
df = df[df['label'].isin(valid_labels)]


In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# ✅ Load your labeled data
df = pd.read_csv("manual_label_dataset.csv")

# ✂️ Clean label column: remove whitespaces + tabs
df["label"] = df["label"].str.strip()

# ⚠️ Remove labels with less than 2 examples (to allow stratified split)
label_counts = df["label"].value_counts()
valid_labels = label_counts[label_counts >= 2].index
df = df[df["label"].isin(valid_labels)]

print("✅ Loaded rows:", len(df))
print("📊 Label distribution:\n", df["label"].value_counts())

# 🎯 Encode labels
label_encoder = LabelEncoder()
df["encoded_label"] = label_encoder.fit_transform(df["label"])

# 🔁 Text and labels
texts = df["user_story"].tolist()
labels = df["encoded_label"].tolist()

# ✂️ Train/Val split (stratified)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

# 🔠 Tokenizer and model
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# 🧱 Dataset class
class UserStoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = UserStoryDataset(train_encodings, train_labels)
val_dataset = UserStoryDataset(val_encodings, val_labels)

# 🧠 Model
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {i: label for label, i in label2id.items()}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# ⚙️ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none"  # 📴 Disable W&B to avoid crashing
)

# 🧪 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# 🚀 Train!
trainer.train()


✅ Loaded rows: 241
📊 Label distribution:
 label
Task Management    33
E-commerce         27
Banking            24
Other              18
Jobs               16
Fitness            15
Parking            12
News               11
Weather            10
Books              10
Social Media       10
RentalCars          9
Travel              8
Education           7
Events              6
Music               5
Billing             4
Reviews             4
Accounts            4
Promotions          3
Ordering            3
Account             2
Name: count, dtype: int64


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-61-2949811869.py:86: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.950100,2.833929
2,2.661900,2.460724
3,2.168300,2.062253
4,1.824900,1.813214
5,1.568900,1.739613


TrainOutput(global_step=120, training_loss=2.2634011348088583, metrics={'train_runtime': 143.0528, 'train_samples_per_second': 6.711, 'train_steps_per_second': 0.839, 'total_flos': 11429388218880.0, 'train_loss': 2.2634011348088583, 'epoch': 5.0})

In [62]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")


('my_model/tokenizer_config.json',
 'my_model/special_tokens_map.json',
 'my_model/vocab.txt',
 'my_model/added_tokens.json',
 'my_model/tokenizer.json')

In [64]:
import pickle

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


In [65]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr
import pickle

# 🔓 Load model + tokenizer
model = AutoModelForSequenceClassification.from_pretrained("my_model")
tokenizer = AutoTokenizer.from_pretrained("my_model")

# 🔓 Load label encoder
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# 🧠 Prediction function
def predict_domain(user_story):
    inputs = tokenizer(user_story, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    return predicted_label

# 🖼️ Gradio UI
iface = gr.Interface(
    fn=predict_domain,
    inputs=gr.Textbox(label="📝 Enter a User Story"),
    outputs=gr.Textbox(label="🔮 Predicted Domain"),
    title="📌 User Story Domain Classifier",
    description="Paste a user story to see what domain it belongs to. Trained using DistilBERT."
)

# 🚀 Launch the app
iface.launch(share=True)  # Use share=True to get a public link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://35d8a77aadf3ff6edf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [66]:
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")


('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [67]:
!zip -r saved_model.zip saved_model


  adding: saved_model/ (stored 0%)
  adding: saved_model/special_tokens_map.json (deflated 80%)
  adding: saved_model/config.json (deflated 57%)
  adding: saved_model/tokenizer.json (deflated 71%)
  adding: saved_model/vocab.txt (deflated 53%)
  adding: saved_model/tokenizer_config.json (deflated 74%)
  adding: saved_model/model.safetensors (deflated 8%)


In [68]:
files.download("saved_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

files.download("label_encoder.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>